#Import/Clean

##Download the file to Colab to begin.
"Independent_Medical_Review__IMR__Determinations__Trend.csv"

In [ ]:
import pandas as pd

In [ ]:
def wrangle(csv) :
  data=pd.read_csv(csv)
  df=pd.DataFrame(data)
  #clean column names
  df = df.rename(columns=str.lower)
  df.columns = df.columns.str.replace(" ","_")
  #sort by report year
  df.sort_values(by='report_year',ascending=False,inplace=True)
  #drop rows with null values
  df.dropna(inplace=True)
  df.drop("reference_id",axis=1,inplace=True)
  return df

In [ ]:
df=wrangle("/content/Independent_Medical_Review__IMR__Determinations__Trend.csv")

FileNotFoundError: ignored

In [ ]:
df.info()

#EDA

In [ ]:
# EDA
df.columns

In [ ]:
type(df["report_year"][0])

In [ ]:
df["diagnosis_category"].value_counts()

In [ ]:
df["treatment_sub_category"].value_counts()

In [ ]:
df["treatment_category"].value_counts()

In [ ]:
df["type"].value_counts()

In [ ]:
df["determination"].value_counts()


In [ ]:
#Read random observation from 'finding'
df["findings"][1030]

In [ ]:
#Let us find the percentage of cases ('type') is overturned or upheald by diagnosis_category.
df.columns

In [ ]:
chance_of_overturn=[]

for i in df["diagnosis_category"].unique():
  a=df["determination"].loc[df["diagnosis_category"]==i]
  if "Overturned Decision of Health Plan" in list(a):
    chance_of_overturn.append([i,a.value_counts()["Overturned Decision of Health Plan"]/len(a)])
    continue
  chance_of_overturn.append([i,0.0])

chance_of_overturn=pd.DataFrame(chance_of_overturn).sort_values(by=1,ascending=False).reset_index(drop=True)
chance_of_overturn.columns=["diagnosis_category","chance_of_overturn"]
#chance_of_overturn
chance_of_overturn

In [ ]:
#Has overturning changed throughout the years?
by_year=df[['report_year','determination']]

In [ ]:
years=df["report_year"].unique()
years

In [ ]:
#Has overturning changed throughout the years?
years=df["report_year"].unique()
results=[]
for i in years:
  year_count=df['determination'].loc[df['report_year']==i]
  results.append([i,(year_count.value_counts()["Overturned Decision of Health Plan"]/len(year_count))])

year_counts=pd.DataFrame(results,columns=["year","overturn_rate"])

In [ ]:
year_counts.sort_values("year")

#Model

In [ ]:
#Things to model
#Does length of findings description influence outcome?
#Can one use a tree model to predict determination outcome?
#Are certain subtypes of diagnosis more likely to produce an overturn?
#Can we locate keywords for certain deseases to determine if they contribue to overturning the case?
#Has the chance of overturning a certain desease changed over the years?
#Can we predict subtype of diagnosis?

#Does age determine what type of diagnosis is given?

#

In [ ]:
#Can one use a tree model to predict determination outcome?

In [ ]:
!pip install category_encoders

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score 

df.drop(['treatment_sub_category','diagnosis_sub_category','type','findings'],axis=1,inplace=True)

In [ ]:
#Create Baseline
df["determination"].value_counts()[0]/len(df)

##Baseline
0.5585397125312099

In [ ]:
df.columns

In [ ]:
from category_encoders import OrdinalEncoder

In [ ]:
df.sort_values('report_year',ascending=False,inplace=True)

y=df["determination"]
X=df.drop("determination",axis=1)
#Split Data by cuttoff date.
cuttoff=df["report_year"]<=2013
X_train=X[cuttoff]
X_test=X[~cuttoff]
y_train=y[:len(X_train)]
y_test=y[len(X_train):]
len(X_train),len(X_test),len(y_train),len(y_test)

In [ ]:
oe=OrdinalEncoder()

y_train=oe.fit_transform(y_train)
y_test=oe.fit_transform(y_test)


X_train=oe.fit_transform(X_train,labels=['diagnosis_category','treatment_category', 'age_range', 'patient_gender'])
X_test=oe.fit_transform(X_test,labels=['diagnosis_category','treatment_category', 'age_range', 'patient_gender'])

X_train

In [ ]:
model=RandomForestClassifier()
model.fit(X_train,y_train)


In [ ]:
from sklearn.metrics import confusion_matrix
y_pred=model.predict(X_test)
accuracy_score(y_test, y_pred),confusion_matrix(y_test, y_pred)


##RandomForrestClassifier
0.5190015740948954

#Tune Model

In [ ]:
!pip install category_encoders
from category_encoders import OrdinalEncoder

In [ ]:
from tables.file import parameters
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from category_encoders import OrdinalEncoder
df=wrangle("/content/Independent_Medical_Review__IMR__Determinations__Trend.csv")
df.sort_values('report_year',ascending=True,inplace=True)
df.drop(['findings'],axis=1,inplace=True)

###
'''
['report_year', 'diagnosis_category', 'diagnosis_sub_category',
'treatment_category', 'treatment_sub_category', 'type', 'age_range',
'patient_gender', 'findings']
'''
###tts
y=df["determination"]
X=df.drop("determination",axis=1)

#Split Data by cuttoff date.
cuttoff=df["report_year"]<=2013
X_train=X[cuttoff]
X_test=X[~cuttoff]
y_train=y[:len(X_train)]
y_test=y[len(X_train):]
###
ohe=OneHotEncoder()

X_train=ohe.fit_transform(X_train)
X_test=ohe.fit_transform(X_test)
###
model=RandomForestClassifier()

###model
from sklearn.model_selection import GridSearchCV
parameters={'n_estimators':[300],'max_depth':[3,5,7],'min_samples_split':[2,6,12]}
clf = GridSearchCV(model, parameters, scoring='accuracy',error_score='raise')

clf.fit(X_train,y_train)

clf.best_params_,clf.best_score_

In [ ]:
X_train["treatment_sub_categ"]

In [ ]:
!pip install category_encoders
from category_encoders import OrdinalEncoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 3.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from tables.file import parameters
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from category_encoders import OrdinalEncoder
from sklearn.model_selection import RandomizedSearchCV
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
# import data
df = pd.read_csv("/content/train.csv")

# check dataset

# taking numerical data

# analyis numerical data

# check null value

# drop null
# now split data
X = df.drop("SalePrice",axis=1)
y = df['SalePrice']
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.25)

# When use pipeline class
pipeline = Pipeline([("si",SimpleImputer),
    ("Encoder",OrdinalEncoder()),
    ("rf", RandomForestClassifier())])
# import Grid Search class
from sklearn.model_selection import GridSearchCV
# make lists of different parameters to check
parameters = {
  'rf__min_samples_split': [2,3,4,5]
  }
  
# initialize
#grid_pipeline = GridSearchCV(pipeline,parameters)

pipe1=Pipeline([("oe", OrdinalEncoder()),
                ("si", SimpleImputer()),
                ("rf", RandomForestClassifier())])

# fit
#X_train_encoded = OrdinalEncoder().fit_transform(X_train)
#X_train_comp = SimpleImputer().fit_transform(X_train_encoded)

clf = RandomizedSearchCV(estimator = pipe1, 
                         param_distributions= parameters)
clf.fit(X_train,y_train)

#grid_pipeline.fit(X_train,y_train)
#grid_pipeline.best_params_

RandomizedSearchCV(estimator=Pipeline(steps=[('oe', OrdinalEncoder()),
                                             ('si', SimpleImputer()),
                                             ('rf', RandomForestClassifier())]),
                   param_distributions={'rf__min_samples_split': [2, 3, 4, 5]})

In [ ]:
clf.best_params_

{'rf__min_samples_split': 3}

In [ ]:
# Use this cell to experiment and then change 
# your model hyperparameters in Task 6
imputer = SimpleImputer()
encoder = OrdinalEncoder()

X_train_encoded = encoder.fit_transform(X_train)
X_train_comp = imputer.fit_transform(X_train_encoded)

param_grid = {'n_estimators': range(200,2000, 10),
              'max_depth' : range(10,110, 10),
              'min_samples_split' : [2,5,10],
              'min_samples_leaf' : [1,2,5]}

clf = RandomizedSearchCV(estimator = RandomForestClassifier(), 
                         param_distributions= param_grid)
clf.fit(X_train_comp,y_train)

In [ ]:
df = pd.read_csv("/content/train.csv")
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125
